# Trading News and Corporate Actions with NLP

<a href="" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


<a href="" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Algo-trading on corporate actions by leveraging NLP. A replicationa and enhancement of the paper: *Trade the Event: Corporate Events Detection for News-Based Event-Driven Trading (Zhou et al., Findings 2021)*.

We will perform the following steps:
1. Domain adaptation for financial articles by finetuning a BERT model with Masked Language Model (MLM) training on financial news and encyclopedia data. *Zhou et al.* utilized human annotators to label news articles with an event. 
1. Bi-Level Event Detection: At Token-Level we detect events using a sequence labeling approach. At the higher Article-Level we will augment the corpus with 'CLS' token embedding which contains the the aggregate of all the article's embeddings, and concatenate it with the lower level tokens.
1. Recognize security Ticker, using string matching algorithm to recognize tickers within articles.
1. Create trading signals on the identified tickers.

```bibtex
@inproceedings{zhou-etal-2021-trade,
    title = "Trade the Event: Corporate Events Detection for News-Based Event-Driven Trading",
    author = "Zhou, Zhihan  and
      Ma, Liqian  and
      Liu, Han",
    editor = "Zong, Chengqing  and
      Xia, Fei  and
      Li, Wenjie  and
      Navigli, Roberto",
    booktitle = "Findings of the Association for Computational Linguistics: ACL-IJCNLP 2021",
    month = aug,
    year = "2021",
    address = "Online",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/2021.findings-acl.186",
    doi = "10.18653/v1/2021.findings-acl.186",
    pages = "2114--2124",
}
```

In [ ]:
"""
%pip install matplotlib
%pip install tensorflow
%pip install shutil
%pip install yfinance
%pip install pyarrow
%pip install tqdm
%pip install pip install transformers
"""

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    print('Running in Kaggle...')
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

    DATA_PATH = "/kaggle/input/DATASET"
    IN_KAGGLE = True
else:
    IN_KAGGLE = False
    DATA_PATH = "./data/"

In [ ]:
import numpy as np
import math
import shutil
import yfinance as yf
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from datetime import datetime
import tensorflow as tf

from tqdm import tqdm

if ('COLAB_TPU_ADDR' in os.environ and IN_COLAB) or (IN_KAGGLE and 'TPU_ACCELERATOR_TYPE' in os.environ):
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
elif len(tf.config.list_physical_devices('GPU')) > 0:
    strategy = tf.distribute.MirroredStrategy()
    gpus = tf.config.experimental.list_physical_devices('GPU')
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
    finally:
        print("Running on", len(tf.config.list_physical_devices('GPU')), "GPU(s)")
else:
    strategy = tf.distribute.get_strategy()
    print("Running on CPU")

print("Number of accelerators:", strategy.num_replicas_in_sync)
os.getcwd()

# Wrangling the Data

Our corpus will be processed and labelled to 11 types of corporate events: 
1. Acquisition(A)
1. Clinical Trial(CT)
1. Regular Dividend(RD)
1. Dividend Cut(DC)
1. Dividend Increase(DI)
1. Guidance Increase(GI)
1. New Contract(NC)
1. Reverse Stock Split(RSS)
1. Special Dividend(SD)
1. Stock Repurchase(SR)
1. Stock Split(SS).
1. No event (O)

Articles are structured as follows:

```json
'title': 'Title',
'text': 'Text Body',
'pub_time': 'Published datetime',
'labels': {
    'ticker': 'Security symbol',
    'start_time': 'First trade after article published',
    'start_price_open': 'The "Open" price at start_time',
    'start_price_close': 'The "Close" price at start_time',
    'end_price_nday': 'The "Close" price at the last minute of the following 1-3 trading day. If early than 4pm ET its the same day. Otherwise, it refers to the next trading day.', 
    'end_time_1-3day': 'The time corresponds to end_price_1day',
    'highest_price_nday': 'The highest price in the following 1-3 trading', 
    'highest_time_nday': 'The time corresponds to highest_price_1-3day',
    'lowest_price_nday': 'The lowest price in the following 1-3 trading day',
    'lowest_time_nday': 'The time corresponds to lowest_price_1-3day',
}
```

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense
import tensorflow_text as text
from transformers import TFBertModel, BertConfig, BertTokenizerFast, TFBertModel, BertTokenizer, TFBertForTokenClassification, BertConfig, create_optimizer
from sklearn.metrics import classification_report, precision_recall_fscore_support

MODEL_DIR = 'google-bert/bert-base-cased'
MAX_LEN = 256
SPECIAL_TOKEN = '[CLS]'

### Tokenizing News Text

The text body is tokenized, a simple example is shown below, including how the '[CLS]' token is leveraged.

Transformers use self-attention mechanisms represent interactions amongst tokens and their contextual information in the input sequence as a weighted-sum. With this mechanism higher layers of the network will aggregate information from all other tokens in the sequence, in our case '[CLS]' will have such information. 


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(MODEL_DIR)
model = TFBertModel.from_pretrained(MODEL_DIR)

text = "Further this conclusion is the fact that cumulative raw returns increase and exceed the market over the same period. When taken as a whole, the evidence suggests Cramer recommends “hot” stocks, lending credence to the Hot Hand Fallacy in this context. "

inputs = tokenizer.encode_plus(f"{SPECIAL_TOKEN} {text}", return_tensors="tf")
outputs = model(inputs['input_ids'])
hidden_state = outputs.last_hidden_state
embedding = hidden_state[:, 0, :]

print(f"Special Token: {SPECIAL_TOKEN} embedding: {embedding}")

inputs

In [ ]:
from pathlib import Path
import re
import pickle
from copy import deepcopy

def read_wnut(file_path):
    """
    Reads a WNUT-format file to extract and separate tokens and their NER tags.
    The WNUT-format expects each token and its corresponding tag to be separated by a tab ('\t'),
    with each token-tag pair on a new line. Different documents or sentences are separated by
    a blank line.

    Args:
        file_path (str): The path to the WNUT-format file containing the annotated text.

    Returns:
        tuple: A tuple containing two lists:
            - token_docs: A list where each element is a list of tokens from a single document/sentence.
            - tag_docs: A list where each element is a list of tags corresponding to the tokens in the `token_docs`.
    """
    print(file_path)
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

def encode_tags(tags, encodings, tag2id):
    """
    Aligns NER tags with the tokenized input for models employing subword tokenization, such as BERT, that requires adjusting the original word-level tags to match the tokenized structure.
    Note the ignore value (-100, from  PyTorch's CrossEntropyLoss) used for tokens that should not be directly associated with a tag from the original dataset, such as special tokens added during tokenization ([CLS], [SEP], [PAD]).

    Args:
        tags (list of list of str): The original list of tag sequences for each document.
        encodings (transformers.BatchEncoding): The tokenized input data with `offset_mapping` included.
        tag2id (dict): A mapping from tag names to unique integer IDs.

    Returns:
        list of list of int: A list of adjusted label sequences for each document, aligned with the tokenized input.
    """
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        doc_enc_labels = np.ones(len(doc_offset), dtype=int) * -100
        arr_offset = np.array(doc_offset)
        max_len = len(doc_enc_labels[(arr_offset[:, 0] == 0) & (arr_offset[:, 1] != 0)])
        doc_enc_labels[(arr_offset[:, 0] == 0) & (arr_offset[:, 1] != 0)] = doc_labels[:max_len]
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

def load_and_cache_dataset(path=DATA_PATH, model=MODEL_DIR, max_len=MAX_LEN, num_labels=12, cache=True):
    """
    Preprocesses and caches the NER dataset for training with a BERT model.

    Loads training and testing data withs sentences and their corresponding NER tags. It consolidates all unique NER tags from both datasets into a tag2id dictionary, mapping each tag to a unique integer, and its inverse id2tag.
    Computes sequence-level labels for both datasets, where each sequence is represented by a binary label vector indicating the presence of specific tags and Tokenizes and encodes the text data, adjusting for BERT's subword tokenization and removing offset mappings as they're not used in model training.

    NER tags are adjusted to align with the tokenizer's output, transforming them into a numerical format suitable for training.
    Args:
        path (str): Path to the dataset directory.
        model (str): Directory of the pretrained BERT model to use for tokenization.
        max_len (int): Maximum sequence length for tokenization.
        num_labels (int): Number of unique labels in the dataset.
        cache (bool): Caches output, defaults to True.

    Returns:
        Tuple: Processed training and testing encodings, sequence labels, and NER labels.
    """
    def generate_sequence_labels(tags_list, tag2id, num_labels):
        """
        Generates sequence labels for a list of tag sequences.

        Args:
            tags_list (list of list of str): List of tag sequences.
            tag2id (dict): Mapping from tag names to unique integer IDs.
            num_labels (int): Number of unique labels in the dataset.

        Returns:
            list of list of int: Sequence labels for each tag sequence.
        """
        seq_labels = []
        for tags in tags_list:
            tag_set = set(tags)
            current_label = np.zeros([num_labels])
            if len(tag_set) == 1:
                current_label[tag2id['O']] = 1
            else:
                tag_set.discard('O')
                for tag in tag_set:
                    current_label[tag2id[tag]] = 1
            seq_labels.append(list(current_label))
        return seq_labels

    cache_file = os.path.join(path, f'cached_train_test_{max_len}.pkl')

    if cache and os.path.exists(cache_file):
        print("Loading dataset from cache")
        with open(cache_file, 'rb') as f:
            train_encodings, train_seq_labels, train_ner_labels, test_encodings, test_seq_labels, test_ner_labels = pickle.load(f)

        return train_encodings, train_seq_labels, train_ner_labels, test_encodings, test_seq_labels, test_ner_labels

    tokenizer = BertTokenizerFast.from_pretrained(model)
    train_ner_texts, train_ner_tags = read_wnut(os.path.join(path, 'Event_detection/train.txt'))
    test_ner_texts, test_ner_tags = read_wnut(os.path.join(path, 'Event_detection/dev.txt'))
    tags = deepcopy(train_ner_tags)
    tags.extend(test_ner_tags)

    unique_tags = list(set(tag for doc in train_ner_tags for tag in doc))
    tag2id = {tag: id for id, tag in enumerate(sorted(unique_tags))}
    print(f"Tags to identify: {tag2id}")
    id2tag = {id: tag for tag, id in tag2id.items()}

    train_seq_labels = []
    test_seq_labels = []
    train_seq_labels = generate_sequence_labels(train_ner_tags, tag2id, num_labels)
    test_seq_labels = generate_sequence_labels(test_ner_tags, tag2id, num_labels)

    train_encodings = tokenizer(train_ner_texts,
                                is_split_into_words=True,
                                return_offsets_mapping=True,
                                padding='max_length',
                                truncation=True,
                                max_length=max_len)
    test_encodings = tokenizer(test_ner_texts,
                            is_split_into_words=True,
                            return_offsets_mapping=True,
                            padding='max_length',
                            truncation=True,
                            max_length=max_len)

    train_ner_labels = encode_tags(train_ner_tags, train_encodings, tag2id)
    test_ner_labels = encode_tags(test_ner_tags, test_encodings, tag2id)
    train_encodings.pop("offset_mapping")
    test_encodings.pop("offset_mapping")
    if cache:
        data_to_save = (
        train_encodings, train_seq_labels, train_ner_labels, test_encodings, test_seq_labels, test_ner_labels)
        with open(cache_file, 'wb') as f:
            pickle.dump(data_to_save, f)

    return train_encodings, train_seq_labels, train_ner_labels, test_encodings, test_seq_labels, test_ner_labels

train_encodings, train_seq_labels, train_ner_labels, test_encodings, test_seq_labels, test_ner_labels =  load_and_cache_dataset(DATA_PATH)

train_ner_labels

### Attention is all you Need

Vaswani et. al seminal paper *Attention is All You Need*, made self-attention and transformers mainstream.

Self-attention, calculates the relevance of each word in a sentence to every other word. This is done through queries (Q=XW^Q), keys (K=XW^K), and values (V=XW^V) transformed by a learned weight matrix (W) from the input embeddings (X). The attention score between two words is computed by taking the dot product of their queries and keys, followed by a softmax:

$ \text{A}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V $

Where:
- Q represents the queries matrix of current items,
- K represents the keys matrix of items to compare against in the input sequence,
- V represents the values matrix, which are the dot product comparisons between Q and K,
- d_k represents the dimension of the keys and queries,
- A are the Attention wieghts.

In addition, the word embeddings will contain contextual information (dot poduct of A and V), represented as position added to the embedding. This plus the attention wieghts, captures dependencies and relationships.

## BERT Classifier

Built on top of a pretrained BERT (Bidirectional Encoder Representations from Transformers).BERT is an industry tested transformer-based model, pre-trained on a large corpus of text to generate contextual embeddings for input sequences.

We will use a pre-trained Google BERT-Base Cased: with 12-layers + 768-hidden, 12-heads , and 110M parameters. This is the base model used in *Zhou et al. (2021)*.

The architecture can be summarized in 3 componets:
1. Input embeddings + attention masks for the preTrained BERT model. Bert applies transformer blocks with self-attention (attention captures language structures). The model outputs embedding sequences (last layer from BERT NxH) and a pooled summary derived from the first 'CLS' token(a 1XH vector).
1. The sequence outputs (NxH vector) is passed through dense layers and dropouts for the first NER classification, this maps the high-DIM outputs to logits. Padding of unknown tokens helps the model focus on the tasks.
1. NER logits are flattened and concatenated with the pooled summaries to form a new feature vector (NxH + H). The vector is passed again through dense and dropout layers to classify the event as one of the 11 identified (O is ignored).

In [ ]:
class BertForBilevelClassification(tf.keras.Model):
    def __init__(self, config, *args, **kwargs):
        super(BertForBilevelClassification, self).__init__(*args, **kwargs)
        self.num_labels = config.num_labels
        self.max_seq_length = config.max_seq_length

        self.bert = TFBertModel(config)


        self.final_dropout1 = Dropout(config.hidden_dropout_prob)
        self.final_classifier1 = Dense(2048, input_shape=(config.hidden_size + self.max_seq_length * self.num_labels,))
        self.final_dropout2 = Dropout(config.hidden_dropout_prob)
        self.final_classifier2 = Dense(self.num_labels - 1)

        self.seq_dropout = Dropout(config.hidden_dropout_prob)
        self.ner_dropout = Dropout(config.hidden_dropout_prob)

        self.ner_classifier1 = Dense(2048, input_shape=(config.hidden_size,))
        self.ner_classifier2 = Dense(self.num_labels)
        self.dropout1 = Dropout(config.hidden_dropout_prob)
        self.dropout2 = Dropout(config.hidden_dropout_prob)

    def call(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, seq_labels=None, ner_labels=None, output_attentions=None, output_hidden_states=None, return_dict=None, training=False):
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask,
                            inputs_embeds=inputs_embeds,
                            output_attentions=output_attentions,
                            output_hidden_states=output_hidden_states,
                            return_dict=return_dict,
                            training=training)

        sequence_output = self.ner_dropout(outputs[0], training=training)
        ner_logits = self.ner_classifier1(sequence_output)
        ner_logits = self.dropout1(ner_logits, training=training)
        ner_logits = self.ner_classifier2(ner_logits)

        pad_pred = tf.zeros([self.num_labels], dtype=ner_logits.dtype)
        pad_pred[-1] = 1
        ner_logits = tf.where(tf.broadcast_to(attention_mask[:, :, None], ner_logits.shape) == 0, pad_pred, ner_logits)

        final_input = tf.reshape(ner_logits, [ner_logits.shape[0], -1])
        final_input = tf.concat([outputs[1], final_input], axis=1)

        seq_logits = self.final_dropout1(final_input, training=training)
        seq_logits = self.final_classifier1(seq_logits)
        seq_logits = self.final_dropout2(seq_logits, training=training)
        seq_logits = self.final_classifier2(seq_logits)

        outputs = (ner_logits,) + (seq_logits,) + outputs[2:]

        if ner_labels is not None:
            loss_fct = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
            ner_loss = loss_fct(ner_labels, ner_logits)

            seq_loss_fct = tf.keras.losses.BinaryCrossentropy(from_logits=True)
            seq_loss = seq_loss_fct(seq_labels[:, :-1], seq_logits)
            total_loss = ner_loss + seq_loss
            outputs = (total_loss,) + outputs

        return outputs

config = BertConfig.from_pretrained(MODEL_DIR)
config.num_labels = 12
config.max_seq_length = MAX_LEN

# Initialize your custom model
model = BertForBilevelClassification(config)

dummy_input = {'input_ids': tf.keras.Input(shape=(config.max_seq_length,), dtype='int32'),
               'attention_mask': tf.keras.Input(shape=(config.max_seq_length,), dtype='int32')}

# Build model by calling it on dummy input
_ = model(dummy_input)


# Manually load the weights from a pre-trained BERT model, if needed
bert_model = TFBertModel.from_pretrained(MODEL_DIR)
model.bert.set_weights(bert_model.get_weights())

# Now you can compile, fit, or evaluate your model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Conclusion

TODO

## References

- [Zhou, Zhihan, Liqian Ma, and Han Liu. "Trade the event: Corporate events detection for news-based event-driven trading." arXiv preprint arXiv:2105.12825 (2021).](https://aclanthology.org/2021.findings-acl.186)
- [Hugging Face Transformers APIs](https://github.com/huggingface/transformers)
- [Hugging Face Model Repository and Spaces](https://huggingface.co/models)
- [Devlin, Jacob, et al. "Bert: Pre-training of deep bidirectional transformers for language understanding." arXiv preprint arXiv:1810.04805 (2018).](https://arxiv.org/abs/1810.04805)
- [Google Pre-trained BERT Models.](https://github.com/google-research/bert?tab=readme-ov-file)
- [Ashish Vaswani and Noam Shazeer and Niki Parmar and Jakob Uszkoreit and Llion Jones and Aidan N. Gomez and Lukasz Kaiser and Illia Polosukhin, "Attention is All You Need". NIPS (2017)](https://research.google/pubs/attention-is-all-you-need/)

## Github

Article and code available on [Github](https://github.com/adamd1985/news-based-event-driven_algotrading)

Kaggle notebook available [here]()

Google Collab available [here]()

## Media

All media used (in the form of code or images) are either solely owned by me, acquired through licensing, or part of the Public Domain and granted use through Creative Commons License.

## CC Licensing and Use

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.